In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# additional libraries
from string import punctuation
import string
from collections import Counter
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")


In [ ]:
sample_df=pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_df

In [ ]:
print(train_df.info())
print(train_df.describe())

In [ ]:
test_df

In [ ]:
train_df

In [ ]:
#fake disaster tweet
print("fake:", (train_df ['target']== 1).astype(int).sum(axis=0))

In [ ]:
# real disaster tweet
print("real:",(train_df ['target']== 0).astype(int).sum(axis=0))

In [ ]:

# Define the function to remove the punctuation
def remove_punctuations_and_lowercase(text):
    for punctuation in string.punctuation:
        text=text.lower()
        text = text.replace(punctuation, '')
        
    return text
# Apply to the DF series
train_df['text'] = train_df['text'].apply(remove_punctuations_and_lowercase)

In [ ]:
words=' '.join([i for i in train_df['text']]).split()

In [ ]:
counts=Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word:ii for ii,word in enumerate(vocab,1)}

In [ ]:
tweets_ints = []
for review in train_df['text']:
    tweets_ints.append([vocab_to_int[word] for word in review.split()])

In [ ]:
len(tweets_ints)

In [ ]:
# test code
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', tweets_ints[:1])


In [ ]:
# labels to numpy format
target=train_df['target'].to_numpy()
print(target)

In [ ]:
# Before we pad our review text, we should check for reviews of extremely short or long lengths; outliers that may mess with our training.
tweet_lens = Counter([len(x) for x in tweets_ints])
print("Zero-length reviews: {}".format(tweet_lens[0]))
print("Maximum review length: {}".format(max(tweet_lens)))

In [ ]:
print('Number of reviews before removing outliers: ', len(tweets_ints))

## remove any reviews/labels with zero length from the reviews_ints list.
non_zero_index=[ii for ii,item in enumerate(tweets_ints) if len(item)!=0]
tweets_ints = [tweets_ints[ii] for ii in non_zero_index]
encoded_labels = np.array([target[ii] for ii in non_zero_index])

print('Number of reviews after removing outliers: ', len(tweets_ints))

In [ ]:
len(tweets_ints[0])

In [ ]:
def pad_features(tweets_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
#     seq_length=32
    n=[]
## implement function
    for i in tweets_ints:
        if len(i) <= seq_length:
            qz=seq_length-len(i)
            n.append(i+[0]*qz)
        else:
            qz=seq_length-len(i)
            n.append(i[:qz])

    for item in n:
        item.reverse()
    n=np.array(n)    
    
    return n

In [ ]:
seq_length = 32

features = pad_features(tweets_ints, seq_length=seq_length)

In [ ]:
features.shape

# Training, Validation, Test


In [ ]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
split_idx=int(len(features)*0.8)
train_x,remaining_x=features[:split_idx],features[split_idx:]
train_y,remaining_y=encoded_labels[:split_idx],encoded_labels[split_idx:]

test_idx=int(len(remaining_x)*0.5)
val_x,test_x=remaining_x[:test_idx],remaining_x[test_idx:]
val_y,test_y=remaining_y[:test_idx],remaining_y[test_idx:]

## print out the shapes of your resultant feature data

print(train_x.shape)
print(val_x.shape)
print(test_x.shape)

In [ ]:
test_idx

In [ ]:
train_data=TensorDataset(torch.from_numpy(train_x),torch.from_numpy(train_y))
valid_data=TensorDataset(torch.from_numpy(val_x),torch.from_numpy(val_y))
test_data=TensorDataset(torch.from_numpy(test_x),torch.from_numpy(test_y))

batch_size=50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)

In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

# Sentiment Network with PyTorch

In [ ]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

In [ ]:
class TweetLSTM(nn.Module):

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(TweetLSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # define all layers
        self.emb=nn.Embedding(vocab_size,embedding_dim)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim,n_layers,dropout=drop_prob,batch_first=True)

        self.dropout=nn.Dropout(0.3)
        self.fc=nn.Linear(hidden_dim,output_size)
        self.sig=nn.Sigmoid()

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size=x.size(0)
        embeds=self.emb(x)
        lstm_out,hidden=self.lstm(embeds,hidden)

        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out=self.dropout(lstm_out)
        out=self.fc(out)
        sig_out=self.sig(out)

        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = TweetLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

In [ ]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
epochs = 50 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            
torch.save(net.state_dict(), 'model.pth')

In [ ]:
# pd.DataFrame(val_losses).plot()


In [ ]:
# Testing
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)
c=0
net.eval()
# iterate over test data
for inputs, labels in test_loader:
    c+=1
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

In [ ]:
labels.float()

In [ ]:
pred

In [ ]:
correct

In [ ]:
for data in test_loader:
    print(data[2])
#   print("Data: ", data)
#   print("Waveform: {}\nSample rate: {}\nLabels: {}".format(data[0], data[1], data[2]))


In [ ]:
len(data)

In [ ]:
# accident test
test_disaster = 'Number of bodies found after Indonesian earthquake rises to 56'
test_no_disaster='Senate Republicans say Trump should be held accountable for riot -- but not by them'

In [ ]:
def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

In [ ]:

def predict(net, test_review, sequence_length=200):
    ''' Prints out whether a give review is predicted to be 
        positive or negative in sentiment, using a trained model.
        
        params:
        net - A trained net 
        test_review - a review made of normal text and punctuation
        sequence_length - the padded length of a review
        '''
    net.eval()
    test_ints=tokenize_review(test_review)
    features=pad_features(test_ints, seq_length)
    
    feature_tensor=torch.from_numpy(features)
    
    batch_size=feature_tensor.size(0)
    
    h=net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    output,h=net(feature_tensor,h)
    
    pred = torch.round(output.squeeze())
    
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))

    # print custom response
    if(pred.item()==1):
        print("Disaster tweet detected!")
    else:
        print("No disaster tweet detected.")

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [ ]:
sample_submission["target"] = net.predict(test_data)

In [ ]:
seq_length=200
predict(net, test_no_disaster, seq_length)

In [ ]:
# The model outputs logits, we have to apply the sigmoid function and round the result
# prediction = torch.sigmoid(best_model(X_test_t)).data.round_().numpy().flatten()

# Submission
# _test = pd.read_csv("../input/titanic/test.csv")
# submission_df = pd.DataFrame({'PassengerId': _test['PassengerId'], 'Survived': prediction.astype(int)})
# submission_df.to_csv("submission.csv", index=False)

# # Storing the datasets
# train.to_csv("submission_train.csv", index=False)
# test.to_csv("submission_test.csv", index=False)